In [2]:
import pandas as pd
from lazypredict.Supervised import LazyClassifier
from sklearn.model_selection import train_test_split

In [3]:
# import processed file
file = "../data/processed/train_processed.csv"
df = pd.read_csv(file)

df.head()

,international_plan,voice_mail_plan,number_vmail_messages,total_intl_minutes,total_intl_calls,total_intl_charge,number_customer_service_calls,churn,total_net_minutes,total_net_calls,total_net_charge
0,0,1,26,13.70,3.00,3.70,1.00,0,611.50,329.00,55.54
1,0,0,0,12.20,5.00,3.29,0.00,0,527.20,328.00,59.00
2,1,0,0,6.60,7.00,1.78,2.00,0,560.45,248.00,65.22
3,1,0,0,10.10,3.00,2.73,3.00,0,501.90,356.00,49.36
4,0,1,24,7.50,7.00,2.03,3.00,0,766.35,314.00,75.18


In [5]:
# split data
X = df.drop(columns=['churn'])
y = df['churn']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [7]:
# using lazypredict to get a quick overview of the performance of different models
clf = LazyClassifier(verbose=0, ignore_warnings=True, custom_metric=None)
models, predictions = clf.fit(X_train, X_test, y_train, y_test)
model_dictionary = clf.provide_models(X_train, X_test, y_train, y_test)
models

 97%|█████████▋| 31/32 [00:05<00:00,  7.50it/s]

[LightGBM] [Info] Number of positive: 403, number of negative: 2572
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000454 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1010
[LightGBM] [Info] Number of data points in the train set: 2975, number of used features: 10
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.135462 -> initscore=-1.853503
[LightGBM] [Info] Start training from score -1.853503


100%|██████████| 32/32 [00:05<00:00,  5.75it/s]


,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
LGBMClassifier,0.97,0.92,0.92,0.97,0.31
RandomForestClassifier,0.97,0.92,0.92,0.97,0.85
BaggingClassifier,0.97,0.91,0.91,0.96,0.20
DecisionTreeClassifier,0.95,0.91,0.91,0.95,0.04
ExtraTreesClassifier,0.97,0.90,0.90,0.96,0.51
ExtraTreeClassifier,0.91,0.81,0.81,0.91,0.03
SVC,0.94,0.81,0.81,0.93,0.29
LabelPropagation,0.90,0.78,0.78,0.89,0.67
LabelSpreading,0.90,0.78,0.78,0.89,1.03


In [8]:
# using the best model from dictionary
rf_model = model_dictionary['LGBMClassifier']
rf_model.fit(X_train, y_train)
rf_model.score(X_test, y_test)

[LightGBM] [Info] Number of positive: 403, number of negative: 2572
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000352 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1010
[LightGBM] [Info] Number of data points in the train set: 2975, number of used features: 10
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.135462 -> initscore=-1.853503
[LightGBM] [Info] Start training from score -1.853503


0.9725490196078431

In [12]:
from sklearn.metrics import accuracy_score, classification_report

# predict the test data
y_pred = rf_model.predict(X_test)
proba = rf_model.predict_proba(X_test)

# checking the classification report
print(accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

0.9725490196078431
              precision    recall  f1-score   support

           0       0.97      0.99      0.98      1080
           1       0.96      0.86      0.91       195

    accuracy                           0.97      1275
   macro avg       0.97      0.92      0.94      1275
weighted avg       0.97      0.97      0.97      1275

